In [ ]:
# Setup env varianble, if PySpark used localy.
import os
os.environ['SPARK_HOME'] = "/usr/local/spark"
os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'lab'
os.environ['PYSPARK_PYTHON'] = 'python'

In [ ]:
# Apache Spark is an open-source, distributed computing system that provides an efficient and fast processing engine for large-scale data processing tasks (big data processing) 
# One of its main characteristics is its ability to perform in-memory processing, reducing the need to read and write to disk and thereby improving overall performance. _
# Spark supports various programming languages, including Scala, Java, Python, and R, making it versatile for different use cases. 
# Its core abstraction is the Resilient Distributed Dataset (RDD), a fault-tolerant collection of elements that can be processed in parallel.

In [ ]:
# SparkSession which is an entry point to the PySpark application

# SparkSession was introduced in version 2.0
# It’s object spark, an entry point to underlying PySpark functionality in order to programmatically create PySpark RDD, DataFrame. 
# Since 2.0 SparkSession can be used in replace with SQLContext, HiveContext, and other contexts defined prior to 2.0.
# SparkSession is a combined class for all different contexts we used to have prior to 2.0 release (SQLContext and HiveContext etc)

# You can create MULTIPLE SparkSession objects but only ONE SparkContext per JVM. 
# In case you want to create another new SparkContext you should stop the existing Sparkcontext using stop() function before creating a new one.

# SparkSession internally creates SparkConfig and SparkContext with the configuration provided with SparkSession.

# Import SparkSession
from pyspark.sql import SparkSession

# Create SparkSession, using the builder() method
spark = SparkSession.builder \
      .master("local[1]") \
      .appName("DataManipulations") \
      .getOrCreate() 

# add configs to SparkSession
# .config("spark.some.config.option", "config-value")  
# SparkSession with Hive Enable 
# .enableHiveSupport()
# 
# https://spark.apache.org/docs/latest/configuration.html


# Create Another / New SparkSession
# This uses the same app name, master as the existing session. 
# Underlying SparkContext will be the same for both sessions as you can have only one context per PySpark application.
# Many Spark session objects are required when you want to keep Spark tables (relational)

spark2 = SparkSession.newSession
print(spark2)

# Get Existing SparkSession
spark3 = SparkSession.builder.getOrCreate()
print(spark3)

# Once the SparkSession is created, you can add the spark configs during runtime or get all configs.

# Set Config
spark.conf.set("spark.executor.memory", "5g")

# Get a Spark Config
partitions = spark.conf.get("spark.sql.shuffle.partitions")
print(partitions)

# Config information
# https://spark.apache.org/docs/latest/configuration.html

In [ ]:
# Working with Catalogs
# To get the catalog metadata, PySpark Session exposes catalog variable. 
# Note that these methods spark.catalog.listDatabases and spark.catalog.listTables and returns the DataSet.

# Get metadata from the Catalog
# List databases
dbs = spark.catalog.listDatabases()
print(dbs)

# List Tables
tbls = spark.catalog.listTables()
print(tbls)

In [ ]:
# Create Example SparkSession in Apache Spark
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder \
    .appName("MySparkApplication") \
    .config("spark.executor.memory", "2g") \
    .config("spark.sql.shuffle.partitions", "4") \
    .getOrCreate()

# Create your first data frame
data = [("Alice", 25), ("Bob", 30), ("Charlie", 35)]
df = spark.createDataFrame(data, ["Name", "Age"])
df.show()

# Create SparkContext in Apache Spark version 2.x and later
# Get the SparkContext from the SparkSession
sc = spark.sparkContext

# Check if the SparkSession is active
if spark.isActive():
    print("Spark session is active. Closing it.")

# Get the SparkContext associated with the SparkSession
sc = spark.sparkContext

# Check if the SparkContext is active
if not sc.isStopped:
    print("Spark session is active.")
    # Perform your Spark operations here

    # Stop the Spark session
    spark.stop()
    print("Spark session is stopped.")

# To stop SparkSession in Apache Spark, this ensures that resources are properly released and the Spark application terminates correctly
spark.stop()

In [ ]:
# Create Empty RDD / DataFrame manually ( with or without schema (column names-datatypes) )

# Create Empty RDD with no partition   
emptyRDD = spark.sparkContext.emptyRDD()

# Creates Empty RDD using parallelize
rddEmpty= spark.sparkContext.parallelize([])

# Create empty RDD with 10 partitions
rdd = spark.sparkContext.parallelize([],10) 
print("Initial partition count:" + str(rdd.getNumPartitions()))

# If you try to perform operations on empty RDD you going to get ValueError("RDD is empty")

# Create Empty DataFrame with Schema (StructType)

# Create Schema
from pyspark.sql.types import StructType,StructField, StringType
schema = StructType([
  StructField('firstname', StringType(), True),
  StructField('middlename', StringType(), True),
  StructField('lastname', StringType(), True)
  ])

# Create empty DataFrame from empty RDD
df = spark.createDataFrame(emptyRDD, schema)

# Convert empty RDD to Dataframe
dfConvert = emptyRDD.toDF(schema)

# Create empty DataFrame directly.
dfEmpty = spark.createDataFrame([], schema)

# Create empty DatFrame with no schema (no columns)
df3 = spark.createDataFrame([], StructType([]))
df3.printSchema()

dept = [("Finance",10),("Marketing",20),("Sales",30),("IT",40)]
rdd = spark.sparkContext.parallelize(dept)

# Convert PySpark RDD to DataFrame
dfFromRDD1 = rdd.toDF()

# Converts RDD to DataFrame with column names
dfFromRDD2 = rdd.toDF("col1","col2")

deptColumns = ["dept_name","dept_id"]
df2 = rdd.toDF(deptColumns)
df2.show()

# using createDataFrame() - Convert DataFrame to RDD
df = spark.createDataFrame(rdd).toDF("col1","col2")

# Convert DataFrame to RDD
rdd = df.rdd

In [ ]:
# Repartition and Coalesce
reparRdd = rdd.repartition(4)

# DataFrame coalesce
reparRdd = rdd.coalesce(2) 
print(reparRdd.rdd.getNumPartitions())


# RDD Cache - saves RDD computation to storage level `MEMORY_ONLY` meaning it will store the data in the JVM heap
cachedRdd = rdd.cache()

# RDD Persist - used to store the RDD to one of the storage levels
# MEMORY_ONLY,MEMORY_AND_DISK, MEMORY_ONLY_SER, MEMORY_AND_DISK_SER, DISK_ONLY, MEMORY_ONLY_2,MEMORY_AND_DISK_2 

import pyspark
dfPersist = rdd.persist(pyspark.StorageLevel.MEMORY_ONLY)
dfPersist.show(False)

# RDD Unpersist - unpersist() marks the RDD as non-persistent, and remove all blocks for it from memory and disk.
rddPersist2 = dfPersist.unpersist()

In [ ]:
# Since Spark 2.0 SparkSession has become an entry point to PySpark to work with RDD, and DataFrame. 
# Prior to 2.0, SparkContext and RDD used to be an entry point

# DataFrame creation
# The simplest way to create a DataFrame is from a Python list of data. 
# DataFrame can also be created from an RDD and by reading files from several sources.

# Create a simple DataFrame from List
data = [("John", True , 25), ("Alice", False , 30), ("Bob", True , 22)]
columns = ["Name", "Married", "Age"]

# Create DataFrame from List
df = spark.createDataFrame(data = data, schema = columns)

# Create DataFrame from Array of Tuples => RDD => DF
df = spark.createDataFrame(data).toDF(columns)


In [ ]:
# Show the contents of the DataFrame, display DataFrame in a tabular form
df.show()
df.show(2, truncate=4) 
df.show(3, vertical=True) 

# Assuming you are using Databricks notebook
display(df)

# Print the schema of the DataFrame, showing the data types of each column.
df.printSchema()

In [ ]:
# How to access Columns / Rows
# Accessing column by "name" or dot .Name
df.select(df.Age).show()
df.select(df["Age"]).show()

# Accessing column using SQL col() function
from pyspark.sql.functions import col
df.select(col("Age")).show()

# Create DataFrame with Nested Struct using Row class
from pyspark.sql import Row
data = [Row(Name = "Greg", Prop = Row(Hair = "black", eye = "blue")),
      Row(Name = "Ann", Prop = Row(Hair = "grey", eye = "black"))]
df = spark.createDataFrame(data)
df.printSchema()

# Access struct column
df.select(df.Prop.Hair).show()
df.select(df["Prop.Hair"]).show()
df.select(col("Prop.Hair")).show()

# Access all columns from struct
df.select(col("Prop.*")).show()

In [ ]:
# Column Class 
# Column class is a fundamental building block for expressing transformations on DataFrames. 
# It represents a column expression that can be applied to manipulate or transform data within a DataFrame. 
# The Column class provides a wide range of methods and functions that you can use to perform operations on the data in a column.

# Create a Column instance by referencing a column in a DataFrame using square bracket notation (df['columnName']).
from pyspark.sql.functions import col

In [ ]:
# PySpark Column Operators

data=[(100,10,1),(200,20,2),(300,30,3)]
df = spark.createDataFrame(data).toDF("col1","col2","col3")
df.show()

# Arthmetic operations
df.select(df.col1 + df.col2).show()
df.select(df.col1 - df.col2).show() 
df.select(df.col1 * df.col2).show()
df.select(df.col1 / df.col2).show()
df.select(df.col1 % df.col2).show()

df.select(df.col2 > df.col3).show()
df.select(df.col2 < df.col3).show()
df.select(df.col2 == df.col3).show()

In [ ]:
# Generate some random data, multi-column with diffrent data type, to ilustrate data frame oprations and functions.
import random
from datetime import datetime, timedelta
from faker import Faker
fake = Faker()

def generate_sample_data(num_entries=5):
    names = ["Alice", "Bob", "Charlie", "David", "Eva"]
    departments = ["HR", "IT", "Finance", "Marketing", "Operations"]

    current_year = datetime.now().year
    date_of_births = [datetime(current_year - random.randint(10, 50), random.randint(1, 12), random.randint(1, 28)) for _ in range(num_entries)]

    sample_data = []

    for _ in range(num_entries):
        name = fake.first_name()
        age = current_year - date_of_births[_].year
        dept = random.choice(departments)
        salary = random.uniform(40000, 120000)
        dob = date_of_births[_].strftime("%Y-%m-%d")
        married = random.choice([True, False])
        gender = random.choice(["M", "F", None])
        # array of languages spoken
        langs = random.sample(["English", "Spanish", "French", "German", "Chinese"], random.randint(2, 3))
        # contact information (dictionary)
        contact = {'Phone': fake.phone_number(), 'Email': fake.email()}
        # skills (dictionary)
        coms = {'Communication': random.choice(['Excellent', 'Good', 'Average']) }
        
        entry = (name, age, dept, salary, dob, married, gender, langs, contact, coms)
        sample_data.append(entry)

    return sample_data

# Generate sample entries
data = generate_sample_data(100)

In [ ]:
# Class to define the structure of the DataFrame.
# StructField class to define the columns which include column name(String), column type (DataType), nullable column (Boolean) and metadata (MetaData)
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, BooleanType, DateType, ArrayType, MapType

# Define the schema for the DataFrame
schema = StructType([
    StructField('Name', StringType(), True),
    StructField('Age', IntegerType(), True),
    StructField('Dept', StringType(), True),
    StructField('Salary', StringType(), True),
    StructField('Birth', StringType(), True),
    StructField("Married", BooleanType(), True),
    StructField('Gender', StringType(), True),
    StructField('Languages', ArrayType(StringType()), True),
    StructField('Contact', MapType(StringType(), StringType()), True),
    StructField('Skills', MapType(StringType(),StringType()),True)

])

# Create DataFrame
df = spark.createDataFrame(data, schema=schema)
# Show the DataFrame
df.show()
df.printSchema()

# Create df with defined Schema
deptDF = spark.createDataFrame(data, schema)

# Extract field names from the schema and convert to an array
schema_array = [field.name for field in schema.fields]

# Create df with defined column names
deptDF = spark.createDataFrame(data, schema)
deptDF.printSchema()

# Assign 
df = deptDF

In [ ]:
# Checking if a Column Exists in a DataFrame
print(df.schema.fieldNames())

# Check if a column exists
column_to_check = "Age"

if column_to_check in df.schema.fieldNames():
    print(f"The column '{column_to_check}' exists in the DataFrame.")

In [ ]:
# Export schema to Json file
printSchemaJson = deptDF.schema.json()
print(printSchemaJson)

# Load schema from json file
import json
schemaFromJson = StructType.fromJson(json.loads( printSchemaJson) )

# Create df with defined Schema
deptDF = spark.createDataFrame(data, schemaFromJson)
deptDF.printSchema()

In [ ]:
# FUNCTIONS
# PySpark Column Functions
# import functions
from pyspark.sql.functions import *

In [ ]:
# SELECT single, multiple, column by index, column function
# Select is a transformation that returns a new DataFrame and holds the columns that are selected

df.select("Age").show()
df.select(df.Age,df.Name).show()
df.select(df["Married"], df["Birth"]).show()

# By using col() function
from pyspark.sql.functions import col
df.select(col("Languages"),col("Skills")).show()

# Select columns by regular expression
df.select(df.colRegex("`^.*name*`")).show()

# Get all columns from DF
all_columns = df.columns

# Select All columns from List
columns = ["Age", "Name"]
df.select(*columns).show()

# Select All columns
df.select([col for col in df.columns]).show()
df.select("*").show()

#Selects first 3 columns and show top 5 rows
df.select(df.columns[:3]).show(5)

# Selects columns 2 to 4 and show top 5 rows
df.select(df.columns[2:4]).show(5)

In [ ]:
# ALIAS: create an alias for a column. It allows you to rename a column temporarily for a specific operation or query.
# Alias the 'Age' column as 'EmployeeAge'
df_alias = df.select(df['Age'].alias('EmployeeAge'))

# Show the DataFrame with the aliased column
df_alias.show()

# Another Alias example with expresion
df.select(expr(" Name ||','|| Age").alias("FullName") ).show()

In [ ]:
# WithColumn: add a new column or update an existing column in a DataFrame.

# Add a new column "bonus" with a constant value
df = df.withColumn("Bonus", lit(1000))

# Add multiple literal columns
df = df.withColumn("Country", lit("USA")).withColumn("City", lit("NY"))

# Create a Column from an Existing Column
df = df.withColumn("ExtraBonus", col("Salary") * 0.3 )

# Update The Value of an Existing Column
df = df.withColumn("Salary", col("Salary")*0.2)

# Change column DataType
df = df.withColumn("Salary", col("Salary").cast("Integer"))

# Standardize strings (convert to lowercase)
standardized_df = df.withColumn("City", lower(df.City))

# Adding / Changing struct of the DataFrame using struct()
from pyspark.sql.functions import col, struct, when

updatedDF = deptDF.withColumn("OtherInfo", 
                        struct( col("Salary").alias("Salary"),
                                when(col("Salary").cast(IntegerType()) < 2000,"Low")
                                .when(col("Salary").cast(IntegerType()) < 4000,"Medium")
                                .otherwise("High").alias("Salary_Grade")
                        ))\
                    .drop("Skills","Salary")
                              
updatedDF.printSchema()
updatedDF.show(truncate=False)

In [ ]:
# Drop column(s)
df = df.drop("City", "Skills")
df.show()

In [ ]:
# RENAME Column 
df = df.withColumnRenamed("Salary", "Income")

# Rename multiple columns
df = df.withColumnRenamed("Birth", "DateOfBirth").withColumnRenamed("Income","Salary_amount")

# Use toDF() to change all column names.
newColumns = df.columns
df.toDF(*newColumns).printSchema()

# Changing a column name on nested data is not straight forward and requires to create a new schema with new DataFrame columns using StructType 
# and use it using cast function

In [ ]:
# FILTER 
# Filter the rows from RDD/DataFrame based on the given condition or SQL expression
# Both Filter() and Where() functions operate exactly the same

# Filter rows where "age" is greater than 30
df.filter(col("Age") > 30).show()

# Using equals condition
df.filter(df.Country == "USA").show(truncate=False)

# not equals condition
df.filter(df.Country != "USA").show(truncate=False) 
df.filter( ~(df.Country == "USA")).show(truncate=False)

# Using SQL Expression
df.filter("Gender == 'M'").show()

# Not equal
df.filter("Gender != 'M'").show()
df.filter("Gender <> 'M'").show()

# Filter on String Type Column
df.filter(col("Name") == "Alice").show()
# Or 
df.where(df["Name"] == "Alice")

# Filter with Multiple Conditions (AND)
multiple_conditions = df.filter((col("Age") > 30) & (col("Married") == True))

# Filter on Struct Type Column
struct_filter = df.filter(col("Skills.Python") == "Intermediate")
struct_filter.show()

# Filter on Array Type Column
from pyspark.sql.functions import array_contains
df.filter(array_contains(df.Languages,"French")).show()

# Using startswith
df.filter(df.Country.startswith("N")).show()
# Using endswith
df.filter(df.Country.endswith("H")).show()

# Like - SQL LIKE pattern
df.filter(df.Name.like("%rose%")).show()

# rlike - SQL RLIKE pattern (LIKE with Regex)
# This check case insensitive
df.filter(df.Name.rlike("(?i)^*rose$")).show()

# Between - Filter rows where 'Age' is between 20 and 30 - filter rows based on whether a column's values fall within a specified range.
df_between = df.filter(df['Age'].between(20, 30)).show(5)

# Handling outliers (filtering rows with specific conditions)
filtered_outliers_df = df.filter((df.Age >= 18) & (df.Age <= 65))

# Filter rows where 'Name' contains 'on'  -  filter rows based on whether a column contains a specified substring.
df_contains = df.filter(df['Name'].contains('on'))

# Filter rows based on whether a column's values are null or not null.
# Filter rows where 'Dept' is null
df_null = df.filter(df['Dept'].isNull()).show()

# Filter rows where 'Dept' is not null
df_not_null = df.filter(df['Dept'].isNotNull()).show()

# Filter rows where 'Name' starts with 'D'
df_startswith = df.filter(df['Name'].startswith('A'))

# Filter rows where 'Name' ends with 'y'
df_endswith = df.filter(df['Name'].endswith('y'))

# Filter rows where 'Name' is like 'Da%'
df_like = df.filter(df['Name'].like('Da%'))

# Filter rows where 'Name' matches the regular expression 'a$'
df_rlike = df.filter(df['Name'].rlike('a$'))

# Filter rows where 'Dept' is in ['IT', 'HR']
df_isin = df.filter(df['Dept'].isin(['IT', 'HR']))

In [ ]:
# Get Distinct Rows (By Comparing All Columns) 
# Distinct returns a new DataFrame after removing the duplicate records. 
distinctDF = df.distinct()
print("Distinct count: "+ str(distinctDF.count()) )

# Drop duplicate rows explicitly
df_no_duplicates = df.dropDuplicates()
print("Distinct count: "+ str(df_no_duplicates.count()))

# Distinct of Selected Multiple Columns => Drop duplicates on selected columns
dropDisDF = df.dropDuplicates(["Age","Salary"]).show()

In [ ]:
# SORT or OrderBy: 
# sort DataFrame by ascending or descending order based on single or multiple columns.
# By default, it sort/orders by ascending. 
# There is no significant difference in terms of functionality or sorting capability between these two methods. 
# Both can be used to sort rows based on one or more columns in ascending or descending order.

# Sort the DataFrame by the "name" column in ascending order
df_sorted = df.sort("Name", ascending=False).show()

# Sort multiple columns in the DataFrame
df.sort(col("Salary"), col("Name")).show(truncate=False)

# Sort DataFrame with asc/desc functions
df.sort(df.Salary.asc(), df.Name.desc()).show(truncate=False)

# Order the DataFrame by the "age" column in descending order
df_ordered = df.orderBy(col("Age").desc()).show()

# Sort the DataFrame in ascending /descending order based on the 'Age' column
df_asc = df.orderBy(df['Age'].asc())
df_desc = df.orderBy(df['Age'].desc())

# Sort the DataFrame in ascending order based on 'Age', with nulls first
df_asc_nulls_first = df.orderBy(df['Age'].asc_nulls_first())

# Sort the DataFrame in ascending order based on 'Age', with nulls last
df_desc_nulls_last = df.orderBy(df['Age'].desc_nulls_last())

In [ ]:
# GroupBy

from pyspark.sql.functions import mean, min, max, avg, sum

# Create new DataFrames using groupBy and aggregation functions
df_count = df.groupBy("Dept").count()
df_min_salary = df.groupBy("Dept").min("Salary")
df_max_salary = df.groupBy("Dept").max("Salary")
df_avg_salary = df.groupBy("Dept").avg("Salary")
df_mean_salary = df.groupBy("Dept").mean("Salary")

# Group by department and calculate aggregate functions
grouped_df = df.groupBy("Dept").agg(
    mean("Salary").alias("Avg_Salary"),
    min("Salary").alias("Min_Salary"),
    max("Salary").alias("Max_Salary"),
    avg("Age").alias("Avg_Age"),
    sum("Age").alias("Total_Age")
)
# Show the grouped DataFrame
grouped_df.show(truncate=False)

# GroupBy on multiple columns
df.groupBy("Dept", "Age").sum("Salary","Bonus").show()

# GroupBy on multiple columns and perform aggregation
df_grouped = df.groupBy("Dept", "Age").agg({"Salary": "avg", "Name": "count"})
df_grouped.show(truncate=False)


In [ ]:
# Fillna and Fill: 
# Replace missing or null values of a DataFrame. Functionally they both perform same.

# Fill null values in all columns with 0
df_filled = df.fillna(value=0)

# Fill null values in the "age" column with 0 , only Age column 
df_filled = df.fillna(0, subset=["Age"])

# Fill all null values with a default value (e.g., empty string)
df_filled_all = df.fillna("", subset=df.columns)

# replace NULL/None values with an empty string or any constant values
df.na.fill("").show(2)

# Replace column nulls in pipeline
df.na.fill("unknown",["City"]) \
    .na.fill("ND",["Gender"]).show()

# Specify different replacement values for different columns when using fillna()
df.fillna( {"Gender": "NoInfo", "Salary":50000} ).show()

In [ ]:
# MAP transformation that is used to apply the transformation function (lambda) on every element of RDD/DataFrame and returns a new RDD
# the output of map transformations would always have the same number of records as input.

# DataFrame doesn’t have map() transformation to apply the lambda function, so you need to convert the DataFrame to RDD and apply the map() transformation.

# Convert DataFrame to RDD and apply map transformation
rdd_result = df.rdd.map(lambda row: (
    row['Name'].upper(),
    row['Age'] * 2,
    row['Dept'].upper(),
    row['Salary'] * 1.1,  # Increase salary by 10%
    row['Birth'],
    row['Married'],
    row['Gender'],
    ', '.join(row['Languages']),  # Convert list to comma-separated string
    ', '.join([f"{key}: {value}" for key, value in row['Contact'].items()]),  # Convert map to string
    ', '.join([f"{key}: {value}" for key, value in row['Skills'].items()])  # Convert map to string
))

# Convert RDD back to DataFrame
columns_transformed = ["Name", "DoubleAge", "UpperDept", "IncreasedSalary", "Birth", "Married", "Gender", "Languages", "Contact", "Skills"]
df_transformed = rdd_result.toDF(columns_transformed).show(5)

print(df.columns)
# columns = ['Name', 'Age', 'Dept', 'Salary', 'Birth', 'Married', 'Gender', 'Languages', 'Contact', 'Skills', 'Bonus', 'Country', 'City', 'ExtraBonus', 'UpperName', 'ReducedSalary']

# Referring columns by index
rdd_by_index = df.rdd.map(lambda x: (x[0], x[6], x[1]))  # Referring to Name, Gender, and Age by index
df_by_index = rdd_by_index.toDF(["Name", "Gender", "Age"])
df_by_index.show(5)

# Referring column names
rdd_by_names = df.rdd.map(lambda x: (x["Name"], x["Gender"], x["Age"]))  # Referring to Name, Gender, and Age by names
df_by_names = rdd_by_names.toDF(["Name", "Gender", "Age"])
df_by_names.show(5)

# By calling function
def func_to_modify_DF(row):
    name = row.Name
    gender = row.Gender
    age = row.Age * 2
    return (name, gender, age)

rdd_by_function = df.rdd.map(lambda x: func_to_modify_DF(x))
df_by_function = rdd_by_function.toDF(["Name", "Gender", "Age"])
df_by_function.show(5)

In [ ]:
# ForEach action in PySpark is used for performing a function on each element of an RDD or DataFrame. 
# It is often used for side effects, such as printing or saving results.
# foreach() function doesn’t return a value instead it executes the input function on each element of an RDD, DataFrame
# mainly used to print row, to manipulate accumulators, update Kafka topics, and other external sources.

# Print a nice summary for each row
def print_summary(row):
    print(f"{row['Name']} is {row['Age']} years old, {'married' if row['Married'] else 'not married'}, speaks {', '.join(row['Languages'])}, and has skills in {', '.join(row['Skills'].keys())}")

df.foreach(lambda row: print_summary(row))

# Use foreach to update an accumulator count for married individuals
married_accumulator = spark.sparkContext.accumulator(0)

def count_married(row):
    if row['Married'] == True:
        married_accumulator.add(1)

df.foreach(lambda row: count_married(row.asDict()))   # row.asDict() to convert the Row object to a Python dictionary
# Print the count of married individuals ,  # Accessed by driver
print(f"Number of married individuals: {married_accumulator.value}")

# Function to produce each row to Kafka
def produce_to_kafka(row):
    # Assuming you have a Kafka topic named 'employee_topic'
    kafka_bootstrap_servers = 'your_kafka_bootstrap_servers'
    kafka_topic = 'employee_topic'
    kafka_producer_config = {
        'bootstrap.servers': kafka_bootstrap_servers,
        'key.serializer': 'org.apache.kafka.common.serialization.StringSerializer',
        'value.serializer': 'org.apache.kafka.common.serialization.StringSerializer',
    }
    
    # Convert row to JSON string
    row_json = row.toJSON().collect()[0]
    
    # Use Kafka producer to send the row to the topic
    from kafka import KafkaProducer
    producer = KafkaProducer(**kafka_producer_config)
    producer.send(kafka_topic, key=None, value=row_json)
    producer.close()

# Apply the function using foreach
df.foreach(produce_to_kafka)

In [ ]:
# Apply Function to Column

# withColumn(), sql(), select() you can apply a built-in function or custom function to a column. 
# In order to apply a custom function, first you need to create a function and register the function as a UDF

from pyspark.sql.functions import upper

# Apply function using withColumn
df = df.withColumn("UpperName", upper(df["Name"]))

# Apply upper function using select on specific columns
df_upper_select = df.select(
    upper(col("Name")).alias("UpperName_Select"),
    upper(col("Dept")).alias("UpperDept_Select"),
    upper(col("Gender")).alias("UpperGender_Select"),
    "*",
)
df_upper_select.show(truncate=False)

# Custom transformation function
def reduce_salary(df, reduceBy):
    return df.withColumn("ReducedSalary", df["Salary"].cast("float") - reduceBy)

# Apply custom transformation
df = reduce_salary(df, reduceBy=5000)

In [ ]:
# Transform()
# Used to chain the custom transformations and this function returns the new DataFrame after applying the specified transformations.

# Custom transformation 1: Convert "Name" column to uppercase
def to_upper_str_columns(df):
    return df.withColumn("Name", upper(df["Name"]))

# Custom transformation 2: Reduce salary by a specified amount
def reduce_salary(df, reduceBy):
    return df.withColumn("ReducedSalary", df["Salary"] - reduceBy)

# Custom transformation 3: Apply a discount to the new salary
def apply_discount(df, discount):
    return df.withColumn("DiscountedSalary", df["ReducedSalary"] - (df["ReducedSalary"] * discount) / 100)

# Apply transformations using DataFrame.transform()
result_df = df.transform(to_upper_str_columns)\
                .transform(reduce_salary, reduceBy=5000) \
                    .transform(apply_discount, discount = 0.05 )

# Show the resulting DataFrame
result_df.show(truncate=False)

In [ ]:
# Get random sample records from the dataset
# Sample 20% of the DataFrame
df_sampled = df.sample(withReplacement=False, fraction=0.2, seed=42)

# Sample 10% of the DataFrame using "Name" column as the sampling key
df_sampled_by = df.sampleBy(col="Name", fractions={"John": 0.1, "Alice": 0.1}, seed=42).show()

In [ ]:
# User defined function
from pyspark.sql.functions import col, udf
from pyspark.sql.types import FloatType
from pyspark.sql.types import StringType

# Create a udf function by wrapping the above function with udf().
# Convert function to udf

# Create custom function
def upperCase(str):
    return str.upper()

upperCaseUDF = udf(lambda x:upperCase(x),StringType()) 

# Custom UDF with withColumn()
df.withColumn("Diff Name", upperCaseUDF(col("Name"))).show(truncate=False)

# Define a UDF to convert names to uppercase
upper_udf = udf(lambda x: x.upper(), StringType())

# Apply the UDF to the "name" column
df_with_upper_name = df.withColumn("upper_name", upper_udf("Name"))

# Custom function to calculate bonus based on age and salary
@udf(FloatType())
def calculate_bonus(age, salary):
    # You can define your custom logic here
    # Let's say the bonus is 5% of the salary for employees below 30 and 10% for others
    bonus_percentage = 0.05 if age < 30 else 0.1
    return salary * bonus_percentage

# Apply the custom function using withColumn
df_with_bonus = df.withColumn("Bonus", calculate_bonus(df["Age"], df["Salary"]))
df_with_bonus.show(truncate=False)

# Handling inconsistent data using a custom function
def custom_cleaning_function(city):
    if city is None:
        return "Unknown"
    city = city.strip()
    return city

cleaning_udf = udf(custom_cleaning_function, StringType())
cleaned_inconsistent_df = df.withColumn("City", cleaning_udf(df.City))

In [ ]:
# SOME functions from SQL
# Cast the 'Age' column to StringType
df_cast = df.withColumn('Age', df['Age'].cast('string')).printSchema()

# Substr: Extract the first three characters of 'Name'
df_substr = df.withColumn('SubName', df['Name'].substr(1, 3))

# When: Add a new column 'Status' based on the 'Age' column
df_status = df.withColumn('Status', when(df['Age'] < 30, 'Young').otherwise('Old'))
df.select(df.Name, df.Age, when(df.Dept=="IT","programista") \
              .when(df.Dept=="HR","hr") \
              .when(df.Dept==None ,"") \
              .otherwise(df.Dept)\
              .alias("Departmens") \
    ).show()

# PySpark SQL functions lit() and typedLit() are used to add a new column to DataFrame by assigning a literal or constant value 
# Lit is used to create a new column with a literal value.
# Create a new column 'ConstantColumn' with the literal value 42
df_lit = df.withColumn('Constant', lit('cos'))

# Creates a constant literal column
df.withColumn("ConstantColumn", typedLit("ConstantValue"))

# Split: used to split a string column into an array of substrings based on a delimiter.
# Assume 'Name' is a string column
df_split = df.withColumn('NameArray', split('Name', 'a'))
# df_split.show()

# Explode: used to transform an array or map column into multiple rows, duplicating the values of the other columns.
# Assume 'Languages' is an array column
df_explode = df.select('Name', 'Age', 'Dept', 'Birth', explode('Languages').alias('Langs'))

# Array_contains: used to check if a specified value is present in an array column.
# Assume 'Skills' is an array column
df_contains = df.filter(array_contains('Languages', 'German'))

# Array: used to create an array column.
# Create a new array column 'Languages' with values from 'LanguagesSpoken'
df_array = df.withColumn('LanguagesSpoken', array('Languages'))

# Expr: used to parse a SQL expression and return it as a Column
# Create a new column 'TotalAge' as the sum of 'Age' and 5
df_expr = df.withColumn('TotalAge', expr('Age + 5'))

# Regexp_replace: used to replace occurrences of a specified pattern with a replacement string.
# Replace 'o' with '0' in the 'Name' column
df_replace = df.withColumn('NameReplaced', regexp_replace('Name', 'o', '0'))

# Text cleaning with regex
cleaned_text_df = df.withColumn("Name", regexp_replace(df.Name, "[^a-zA-Z0-9 ]", ""))
# df_replace.show()

# Creates a struct column
df.select(struct("Name", "Age", "Dept").alias("EmployeeInfo")).show()

# Collects the elements of a column into a list => collect_list()
df.groupBy("Dept").agg(collect_list("Name").alias("EmployeeList")).show()

# Collect_list is used to aggregate values into a list.
df_collect_list = df.groupBy('Dept').agg(collect_list('Skills').alias('AllSkills'))
# Show the DataFrame with the collected list of skills for each department
df_collect_list.show()

# Collects the unique elements of a column into a set => collect_set
df.groupBy("Dept").agg(collect_set("Languages").alias("UniqueLanguages")).show()
df_collect_set = df.groupBy('Dept').agg(collect_set('Contact').alias('UniqueContact'))


In [ ]:
# MAP
# MapType (also called map type) is a data type to represent Python Dictionary (dict) to store key-value pair

# Create MapType From StructType
from pyspark.sql.types import StructField, StructType, StringType, MapType
schema = StructType([
    StructField('Name', StringType(), True),
    StructField('Properties', MapType(StringType(),StringType()),True)
])

dataDictionary = [
        ('James',{'hair':'black','eye':'brown'}),
        ('Michael',{'hair':'brown','eye':None}),
        ]
df2 = spark.createDataFrame(data=dataDictionary, schema = schema)
df2.printSchema()
df2.show(truncate=False)

# Creates a map from Key-Value pairs. 
# a map of 'Name','Skills' for each entry
df_create_map = df.select("Name", create_map('Name', 'Skills').alias('NameSkillsMap'))
# df_create_map.show(truncate=False)

# Map_keys:  Extracts the keys from a map column, All Map Keys
df.select("Name", map_keys("Contact").alias("SkillKeys")).show()

# Map_values: Extracts the values from a map column, All map Values
df.select("Name", map_values("Skills")).show()

# GetItem is used to extract an element from an array type column.
# Assume 'Languages' is an array type column
df_item = df.select(df['Languages'].getItem(1).alias('LangSkill'))
# Extract values from the "skills" map column
df2 = df2.withColumn("Hairs", col("Properties").getItem("hair")).show()

# GetField is used to extract a field from a struct type column
# 'Contact' is a struct type column with fields 'Phone', 'Email', and 'Address'
df_field = df.select(df['Contact'].getField('Phone').alias('PhoneNumber'))

# Explode data => new row for nested items
df2.select(df2.Name, explode(df2.Properties)).show()

In [ ]:
# Partitioning the data on the file system is a way to improve the performance of the query when dealing with a large dataset
# PySpark partition is a way to split a large dataset into smaller datasets based on one or more partition keys.
# pyspark.sql.DataFrameWriter class which is used to partition the large dataset (DataFrame) into smaller files based on one or multiple columns while writing to disk
# When you create a DataFrame from a file/table, based on certain parameters PySpark creates the DataFrame with a certain number of partitions in memory.
# PySpark supports partition in two ways; partition in memory (DataFrame) and partition on the disk (File system).

# Partition in memory: 
# You can partition or repartition the DataFrame by calling repartition() or coalesce() transformations.
# repartition() creates specified number of partitions in memory. 

# Partition on disk
# Write DataFrame to parquet files partitioned by the "married" column
df.write.partitionBy("Married").mode("overwrite").parquet("output_file_path")

# or CSV writter
df.write.option("header", True) \
        .partitionBy("Married") \
        .mode("overwrite") \
        .csv("married-state")

# partitionBy() Multiple Columns 
df.write.partitionBy("Country", "City").parquet("output_file_path_2")

In [ ]:
# JOINS are used to combine two DataFrames and by chaining these you can join multiple DataFrames
# It supports all basic join type operations available in traditional SQL like INNER, LEFT OUTER, RIGHT OUTER, LEFT ANTI, LEFT SEMI, CROSS, SELF JOIN

# Generate sample data for employees and departments
employees_data = [
    (1, "Alice", 3),
    (2, "Bob", 1),
    (3, "Charlie", 2),
    (4, "David", 2),
]

departments_data = [
    (1, "IT"),
    (2, "HR"),
    (3, "Finance"),
]

# Create DataFrames
employees_df = spark.createDataFrame(employees_data, ['id', 'name', 'manager'])
departments_df = spark.createDataFrame(departments_data, ['id', 'dept'])

# Inner Join: Returns only the rows with matching keys in both DataFrames.
inner_join = employees_df.join(departments_df, "id").show()

# Left Join: Returns all rows from the left DataFrame and matching rows from the right DataFrame.
left_join = employees_df.join(departments_df, employees_df["manager"] == departments_df["id"], "left").show()

# Right Join: Returns all rows from the right DataFrame and matching rows from the left DataFrame.
right_join = employees_df.join(departments_df, "id", "right").show()

# Full Outer Join: Returns all rows from both DataFrames, including matching and non-matching rows.
full_outer = employees_df.join(departments_df, employees_df["manager"] == departments_df["id"], "outer").show()

# Left Semi Join: Returns all rows from the left DataFrame where there is a match in the right DataFrame.
left_semi_join = employees_df.join(departments_df, "id", "left_semi")

# Left Anti Join: Returns all rows from the left DataFrame where there is no match in the right DataFrame.
left_anti_join = employees_df.join(departments_df, "id", "left_anti")

# Self Join on Employee (manager = id)
self_join = employees_df.alias("e1").join(employees_df.alias("e2"), col("e1.manager") == col("e2.id"), "inner").show()

# Cross Join (Cartesian Join - all possible combinations of rows)
cross_join = employees_df.crossJoin(departments_df)

# Join on multiple dataFrames
# df1.join(df2,df1.id1 == df2.id2,"inner") \
#    .join(df3,df1.id1 == df3.id3,"inner")

In [ ]:
# Joins Using SQL Expression

# Create temporary views for your DataFrames
employees_df.createOrReplaceTempView("EMP")
departments_df.createOrReplaceTempView("DEPT")

# Inner Join using SQL Expression
result_inner = spark.sql("SELECT * FROM EMP INNER JOIN DEPT ON EMP.id = DEPT.id")

# Left Join using SQL Expression
result_left = spark.sql("SELECT * FROM EMP LEFT JOIN DEPT ON EMP.id = DEPT.id")

# Right Join using SQL Expression
result_right = spark.sql("SELECT * FROM EMP RIGHT JOIN DEPT ON EMP.id = DEPT.id")

# Full Outer Join using SQL Expression
result_outer = spark.sql("SELECT * FROM EMP FULL OUTER JOIN DEPT ON EMP.id = DEPT.id")

# Left Semi Join using SQL Expression
result_left_semi = spark.sql("SELECT * FROM EMP LEFT SEMI JOIN DEPT ON EMP.id = DEPT.id")

# Left Anti Join using SQL Expression
result_left_anti = spark.sql("SELECT * FROM EMP LEFT ANTI JOIN DEPT ON EMP.id = DEPT.id")

# Cross Join using SQL Expression
result_cross = spark.sql("SELECT * FROM EMP CROSS JOIN DEPT")

# Self Join on Employee using SQL Expression
result_self_join = spark.sql("SELECT * FROM EMP e1 INNER JOIN EMP e2 ON e1.manager = e2.id")

In [ ]:
# UNION
# Union() and UnionAll() transformations are used to merge two or more DataFrame’s of the same schema or structure.

# Generate additional sample data for employees_df for demonstration
additional_employees_data = [
    (5, "Eva", 1),
    (6, "Frank", 3),
    (7, "Grace", 2),
]

# Create a new DataFrame with additional sample data
additional_employees_df = spark.createDataFrame(additional_employees_data, ["id", "name", "manager"])

# Union the two DataFrames
combined_employees_df = employees_df.union(additional_employees_df)
#combined_employees_df.show()

# In PySpark Union does not remove duplicates. Recommend using DataFrame duplicate() function to remove duplicate rows.
# Merge without Duplicates
disDF = employees_df.union(additional_employees_df).distinct()
disDF.show(truncate=False)

# Union two DataFrames with different column order
# unionByName() is used to merge two DataFrames by column names instead of by position.

# Additional sample data with columns in a different order
additional_employees_data = [
    (5, 1, "Eva", "Female"),
    (6, 3, "Frank", "Male"),
    (7, 2, "Grace", "Female"),
]

# Create a new DataFrame with additional sample data and different column order
additional_employees_df = spark.createDataFrame(additional_employees_data, ["id", "gender", "manager", "name"])

# Union the two DataFrames by name
# provides an argument allowMissingColumns to specify if you have a different column counts
combined_employees_df = employees_df.unionByName(additional_employees_df, allowMissingColumns=True)
combined_employees_df.show()

# Union two DataFrames with different column order
df_union_by_name = df.unionByName(df.select("Married", "Name", "Age", "Salary", "Languages", "Skills"), allowMissingColumns=True)
df_union_by_name.show()

In [ ]:
# Pivot is used to rotate/transpose the data from one column into multiple Dataframe columns and back using unpivot(). 
# Pivot() is an aggregation where one of the grouping columns values is transposed into individual columns with distinct data.

# Pivot the DataFrame based on the "Dept" column
pivot_df = df.groupBy("Name").pivot("Dept").agg({"Salary": "max"}).show()

# Unpivot the "Languages" column using explode
unpivot_df = df.select("Name", "Age", explode("Languages").alias("Language")).show()

In [ ]:
# Aggregate Functions

# Count the number of rows
count_rows = df.count()
print(f"Count of rows: {count_rows}")

# Average age
average_age = df.agg(avg("Age").alias("AverageAge"))

# Sum of salaries
total_salary = df.agg(sum("Salary").alias("TotalSalary"))

# Maximum and minimum salary
max_salary = df.agg(max("Salary").alias("MaxSalary"))
min_salary = df.agg(min("Salary").alias("MinSalary"))

# Standard deviation of ages
std_dev_age = df.agg(stddev("Age").alias("StdDevAge"))

# Variance of ages
variance_age = df.agg(variance("Age").alias("VarianceAge"))

# first: Returns the first value in a group/column.
first_value = df.groupBy("Dept").agg(first("Name").alias("FirstEmployee"))
# Last: Returns the last value in a group/column.
last_value = df.groupBy("Dept").agg(last("Name").alias("LastEmployee"))

# First and last names
first_name = df.agg(first("Name").alias("FirstName"))
last_name = df.agg(last("Name").alias("LastName"))

# Approx_count_distinct: Returns the approximate number of distinct values in a column.
approx_distinct_count = df.agg(approx_count_distinct("Name").alias("ApproxDistinctCount"))

# Collect_list: Collects the values of a column into a list.
collected_list = df.groupBy("Dept").agg(collect_list("Name").alias("NameList"))

# Collect_set: Collects the distinct values of a column into a set.
collected_set = df.groupBy("Dept").agg(collect_set("Gender").alias("GenderSet"))

# CountDistinct: Returns the count of distinct values in a column.
distinct_count = df.agg(countDistinct("Dept").alias("DistinctDeptCount"))

In [ ]:
# Window functions

from pyspark.sql.window import Window
from pyspark.sql.functions import col, rank, dense_rank, row_number, sum, avg, max, min


# PySpark Window functions operate on a group of rows (like frame, partition) and return a single value for every input row.
# Window functions are applied over a specific range of rows related to the current row.

# Define a Window specification
window_spec = Window.orderBy(col("Salary"))

# Window specification by department and Order by salary
window_spec_2 = Window.partitionBy("Dept").orderBy("Salary")

# Rank: used to provide a rank to the result within a window partition.
# Rank employees by salary =  assigns a unique rank to each row based on the "Salary" column.
df_rank = df.withColumn("SalaryRank", rank().over(window_spec))

# Dense_rank: used to get the result with rank of rows within a window partition without any gaps.
# The dense_rank function is similar to rank but does not leave gaps between ranks when there are tied values.
df_dense_rank = df.withColumn("SalaryDenseRank", dense_rank().over(window_spec))

# Row_number:  assigns a unique number to each row based on the order specified in the window.
df_row_number = df.withColumn("SalaryRowNumber", row_number().over(window_spec)).show()

# Calculate the cumulative sum of salaries
df_cumulative_sum = df.withColumn("CumulativeSalary", sum(col("Salary")).over(window_spec))

# Calculate the average salary over a rolling window
df_avg_salary = df.withColumn("RollingAvgSalary", avg(col("Salary")).over(window_spec))

# Get the maximum salary for each department
df_max_salary_per_dept = df.withColumn("MaxSalaryPerDept", max(col("Salary")).over(Window.partitionBy("Dept")))

# Get the minimum salary for each department
df_min_salary_per_dept = df.withColumn("MinSalaryPerDept", min(col("Salary")).over(Window.partitionBy("Dept")))

# Lead: returns the value of a given expression for a row at a specified physical offset from that row within the result set.
df_lead = df.withColumn("NextSalary", lead("Salary").over(window_spec_2))

# Lag: returns the value of a given expression for a row at a specified physical offset from that row within the result set.
df_lag = df.withColumn("PrevSalary", lag("Salary").over(window_spec_2))

# Cume Dist: returns the cumulative distribution of a value within a partition of the result set.
df_cume_dist = df.withColumn("CumulativeDistribution", cume_dist().over(window_spec_2))

# Ntile: returns the relative rank of a value within a partition of the result set.
df_ntile = df.withColumn("Ntile", ntile(4).over(window_spec_2))  # Here, 4 represents the number of tiles.

# Percent Rank: returns the relative rank of a value within a partition of the result set, ranging from 0 to 1.
df_percent_rank = df.withColumn("PercentRank", percent_rank().over(window_spec_2))

# Aggregations for each department using window functions

# Avg, Sum, Min, Max, Median, Std for each department
df_aggregations = df.withColumn("AvgSalary", avg("Salary").over(window_spec)) \
    .withColumn("SumSalary", sum("Salary").over(window_spec)) \
    .withColumn("MinSalary", min("Salary").over(window_spec)) \
    .withColumn("MaxSalary", max("Salary").over(window_spec)) \
    .withColumn("MedianSalary", expr("percentile_approx(Salary, 0.5)").over(window_spec)) \
    .withColumn("StdSalary", stddev("Salary").over(window_spec))

# Show the DataFrame with added columns for aggregations
df_aggregations.show(truncate=False)

In [ ]:
# Date and Timestamp Functions
# DateType default format is yyyy-MM-dd 
# TimestampType default format is yyyy-MM-dd HH:mm:ss.SSSS

# Extracting Year, Month, Day from "Birth" column
df_with_date_parts = df.withColumn("Birth_Year", col("Birth").substr(1, 4).cast(IntegerType())) \
                       .withColumn("Birth_Month", col("Birth").substr(6, 2).cast(IntegerType())) \
                       .withColumn("Birth_Day", col("Birth").substr(9, 2).cast(IntegerType()))

# Convert the "Birth" column to a DateType
df = df.withColumn("Birth", to_date(col("Birth"), "yyyy-MM-dd"))

# Convert a string column to a date.
df = df.withColumn('BirthDate', to_date('Birth', 'yyyy-MM-dd'))

# Current date and timestamp
df_with_current_date = df_with_date_parts.withColumn("CurrentDate", current_date())
df_with_current_timestamp = df_with_current_date.withColumn("CurrentTimestamp", current_timestamp())

# Adding and subtracting days from "Birth" column
df_with_added_days = df_with_current_timestamp.withColumn("BirthPlus10Days", date_add(col("Birth"), 10))
df_with_subtracted_days = df_with_added_days.withColumn("BirthMinus5Days", date_sub(col("Birth"), 5))

# Calculating the difference in days between two dates
df_with_date_diff = df_with_subtracted_days.withColumn("DateDiff", datediff(col("Birth"), col("BirthMinus5Days")))

# Converting string to date
df_with_date_conversion = df_with_date_diff.withColumn("BirthAsDate", to_date(col("Birth"), "yyyy-MM-dd"))

# Converting timestamp to UTC
df_with_utc_timestamp = df_with_date_conversion.withColumn("UTC_Timestamp", to_utc_timestamp(col("CurrentTimestamp"), "GMT"))

# Converting timestamp to Unix timestamp
df_with_unix_timestamp = df_with_utc_timestamp.withColumn("Unix_Timestamp", unix_timestamp(col("CurrentTimestamp")))

# date_format is used to format a date or timestamp column.
# Assume 'BirthDate' is a date column
df_date_format = df.withColumn('FormattedDate', date_format('BirthDate', 'MM/dd/yyyy'))

# Formatting Date
df = df.withColumn("FormattedBirth", date_format("Birth", "MMM dd, yyyy"))

# Calculate Age using datediff
df = df.withColumn("CurrentDate", trunc(current_date(), "yyyy-MM-dd"))

df = df.withColumn("AgeCalculated", datediff("CurrentDate", "Birth") / 365)

# Datediff is used to calculate the difference in days between two date columns.
# Assume 'BirthDate' and 'CurrentDate' are date columns
df_datediff = df.withColumn('DaysSinceBirth', datediff(current_date(), 'BirthDate'))

# Assume 'BirthDate' and 'CurrentDate' are date columns
df_months_between = df.withColumn('MonthsSinceBirth', months_between(current_date(), 'BirthDate'))

# Day of the week
df = df.withColumn("NextMonday", next_day("Birth", "Mon"))

# Week of the year
df = df.withColumn("WeekOfYear", weekofyear("Birth"))

# Day of the week
df = df.withColumn("DayOfWeek", dayofweek("Birth"))

# Day of the month
df = df.withColumn("DayOfMonth", dayofmonth("Birth"))

# Day of the year
df = df.withColumn("DayOfYear", dayofyear("Birth"))

# Convert string to timestamp, 
# Assume 'Birth' is a string column in 'yyyy-MM-dd' format
df = df.withColumn("BirthTimestamp", to_timestamp("Birth", "yyyy-MM-dd"))

# Extract Hour, Minute, and Second from timestamp
df = df.withColumn("Hour", hour("BirthTimestamp"))
df = df.withColumn("Minute", minute("BirthTimestamp"))
df = df.withColumn("Second", second("BirthTimestamp"))

# Show the resulting DataFrame
df.show(5, truncate=False)


In [ ]:
# JSON 
# Parses a JSON string column into a struct or map.
df.select("Name", from_json("Contact", schema_of_json("Contact")).alias("ContactInfo"))

# Converts a struct or map column to a JSON string.
df.select("Name", to_json("Contact").alias("ContactJSON"))

# Extracts values from a JSON string column using specified keys.
df.select("Name", json_tuple("Contact", "Phone", "Email").alias("ContactDetails"))

# Extracts a value from a JSON string column using a JSON path expression.
df.select("Name", get_json_object("Contact", "$.Email").alias("EmailAddress"))

#Infers the schema of a JSON string column.
df.select("Name", schema_of_json("Contact").alias("ContactSchema"))

In [ ]:
# Convert PySpark Dataframe to Pandas DataFrame
pandasDF = deptDF.toPandas()
print(pandasDF)

# toPandas() results in the collection of all records in the PySpark DataFrame to the driver program and should be done only on a small subset of the data
# Pandas API on Spark > 3.2

# DataFrame corresponds to pandas DataFrame logically and it holds Spark DataFrame internally. 
# In other words, it is a wrapper class for Spark DataFrame to behave similarly to pandas DataFrame.

# Import PySpark Pandas
import pyspark.pandas as ps

# Create pandas DataFrame
technologies   = ({
    'Courses':["Spark","PySpark","Hadoop","Python","Pandas","Hadoop","Spark","Python","NA"],
    'Fee' :[22000,25000,23000,24000,26000,25000,25000,22000,1500],
    'Duration':['30days','50days','55days','40days','60days','35days','30days','50days','40days'],
    'Discount':[1000,2300,1000,1200,2500,None,1400,1600,0]
          })
df = ps.DataFrame(technologies)
print(df)

# Use groupby() to compute the sum
df2 = df.groupby(['Courses']).sum()
print(df2)

# Convert this pyspark.pandas.frame.DataFrame object to pandas.core.frame.DataFrame (Convert Pandas API on Spark to Pandas DataFrame)
pdf = df.to_pandas()
print(type(pdf))

# Convert pandas.core.frame.DataFrame to pyspark.pandas.frame.DataFrame (Convert Pandas DataFrame to Pandas API on Spark DataFrame)
psdf = ps.from_pandas(pdf)
print(type(psdf))

# Pandas API on Spark Dataframe into a Spark DataFrame
# Converts object from type pyspark.pandas.frame.DataFrame to pyspark.sql.dataframe.DataFrame.
sdf = df.to_spark()
print(type(sdf))
sdf.show()

# Convert a Spark Dataframe into a Pandas API on Spark Dataframe
# Convert pyspark.sql.dataframe.DataFrame to pyspark.pandas.frame.DataFrame DataFrame
psdf = sdf.pandas_api()
print(type(psdf))
# (or)
# to_pandas_on_spark() is depricated
psdf = sdf.to_pandas_on_spark()
print(type(psdf))

In [ ]:
# Spark SQL brings native RAW SQL queries on Spark meaning you can run traditional ANSI SQL on Spark Dataframe 

# Using SparkSession you can access PySpark/Spark SQL capabilities in PySpark. 
# Allow to run any traditional SQL queries on DataFrame using PySpark SQL.

# In order to use SQL features first, you need to create a temporary view in PySpark. 
# Once you have a temporary view you can run any ANSI SQL queries using spark.sql() met

# Spark SQL
df.createOrReplaceTempView("PERSON_DATA")
df2 = spark.sql("SELECT * from PERSON_DATA")
df2.printSchema()
df2.show()

# Query dataframe by SQL
groupDF = spark.sql("SELECT gender, count(*) from PERSON_DATA group by gender")
groupDF.show()

In [ ]:
# If you want to have a temporary view that is shared among all sessions and keep alive until the Spark application terminates, 
# you can create a global temporary view using createGlobalTempView()

# In PySpark, you can use the createGlobalTempView method to create a global temporary view of a DataFrame. 
# A global temporary view is accessible from any SparkSession within the same Spark application, and it's useful when you want to share data across different SparkSessions or notebooks.

# In this use case, you create a global temporary view, and then in another SparkSession, you can access and query the same global temporary view by specifying the view name as global_temp.<view_name>.

# Create a global temporary view,  Assuming you have a DataFrame named 'df'
df.createGlobalTempView("global_sample_table")

# In another SparkSession or notebook, you can access the global temporary view
df2 = spark.newSession().sql("SELECT _1, _2 FROM global_temp.global_sample_table")
df2.show()

In [ ]:
# SparkSession is used to create and query Hive tables. Note that in order to do this for testing you don’t need Hive to be installed. 
# saveAsTable() creates Hive managed table. Query the table using spark.sql().

# Save the contents of the global temporary view to a persistent table
spark.sql("SELECT * FROM global_temp.global_sample_table").write.saveAsTable("my_persistent_table")

# Create Hive table & query it.  
spark.table("sample_table").write.saveAsTable("sample_hive_table")
df3 = spark.sql("SELECT _1,_2 FROM sample_hive_table")
df3.show()

In [ ]:
# Create a Database CT
spark.sql("CREATE DATABASE IF NOT EXISTS ct")

# Create a Table naming as sampleTable under CT database.
spark.sql("CREATE TABLE ct.sampleTable (id Int, name String, age Int, gender String)")

# Insert into sampleTable using the sampleView. 
spark.sql("INSERT INTO TABLE ct.sampleTable  SELECT * FROM sampleView")

# Lets view the data in the table
spark.sql("SELECT * FROM ct.sampleTable").show()

In [ ]:
# PySpark Streaming
# Streaming is a scalable, high-throughput, fault-tolerant streaming processing system that supports both batch and streaming workloads. It is used to process real-time data from sources like file system folders, TCP sockets, S3, Kafka, Flume, Twitter, and Amazon Kinesis to name a few. 
# The processed data can be pushed to databases, Kafka, live dashboards e.t.c

# Streaming from socket
df = spark.readStream \
      .format("socket") \
      .option("host","localhost") \
      .option("port","9090") \
      .load()

# Streaming from Kafka
df = spark.readStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers", "192.168.1.100:9092") \
        .option("subscribe", "json_topic") \
        .option("startingOffsets", "earliest") \
        .load()

#  Write a message to another topic in Kafka using writeStream()

df.selectExpr("CAST(id AS STRING) AS key", "to_json(struct(*)) AS value") \
   .writeStream \
   .format("kafka") \
   .outputMode("append") \
   .option("kafka.bootstrap.servers", "192.168.1.100:9092") \
   .option("topic", "josn_data_topic") \
   .start() \
   .awaitTermination()

In [ ]:
"""
Spark RDD, which stands for Resilient Distributed Dataset, is a fundamental data structure in Apache Spark, 
a powerful framework for big data processing. 
In simple words, you can think of an RDD as a distributed collection of data that can be processed in parallel across multiple computers.

Imagine you have a massive dataset (like a huge text file or a database) that is too big to fit on a single computer. 
An RDD helps you break that data into smaller, manageable chunks and distribute those chunks across a cluster of computers (nodes). 
Each computer in the cluster can process its own chunk of data independently.

For example, if you have a text file containing information about sales transactions, you can create an RDD from it. 
Each line in the file would become a separate element in the RDD. 
The RDD is divided into partitions, and each partition is processed on a different computer in the cluster.
"""

""" 
RDDs (Resilient Distributed Datasets) are a core concept in Apache Spark, a powerful distributed data processing framework. 
RDDs provide a flexible and fault-tolerant way to process and analyze large datasets in parallel across a cluster of computers. 
Here are some key aspects of RDDs:

- Distributed Data: RDDs represent distributed collections of data, split into smaller partitions, and distributed across multiple nodes in a Spark cluster. 
Each node processes its portion of the data in parallel, making it suitable for big data processing.

- Resilience: The "Resilient" part of RDDs means they are fault-tolerant. If a node fails during processing, 
Spark can recompute the lost data using the lineage information (a record of the transformations applied to the data).
This ensures data consistency and reliability.

- Immutable: RDDs are immutable, meaning their content cannot be changed once created. 
You can transform an RDD into another RDD, but you cannot modify it in place. 
This immutability simplifies distributed data processing because it eliminates the need for complex synchronization mechanisms.

- Lazy Evaluation: RDDs follow a lazy evaluation model. When you perform operations on an RDD, the transformations are not immediately executed. 
Instead, they are recorded as a sequence of transformations (a lineage). Actual computation occurs only when an action is called. 
This allows Spark to optimize the execution plan.

- Resilient Transformations: RDDs support two types of operations:

a) Transformations: These are operations that create a new RDD from an existing one, such as map, filter, reduceByKey, and join. 
Transformations are lazily evaluated.
b) Actions: These are operations that trigger the execution of transformations and return values, such as count, collect, and saveAsTextFile.

RDDs have been a foundational concept in Spark, but it's important to note that as of Spark 2.0 and later, 
DataFrames and Datasets have become more prevalent for structured data processing, offering higher-level abstractions and optimizations. 
Nonetheless, RDDs are still useful for more fine-grained control over data processing and remain a crucial part of the Spark ecosystem.

"""

""" 
Data lineage information in the context of distributed data processing, like in Apache Spark, refers to a record or history of how your data was transformed from its original state to its current form. It traces the sequence of operations or transformations that were applied to the data.

Think of data lineage like a family tree or genealogy chart for your data. It shows the parent-child relationships between different versions of the data, indicating which operations were performed on the data to produce each new version. This lineage information is essential for maintaining data integrity, 
fault tolerance, and ensuring that computations can be re-executed if there's a failure.

In simple words, data lineage answers the question, "How did we get this data?" It's like a step-by-step history that shows how your data evolved and was derived from its original source through various transformations and operations. 
This information is crucial for debugging, auditing, and ensuring that you can recover or reproduce your data and analysis if something goes wrong.

"""